In [1]:
from binary_models.load_ternary import load_community_model
community = load_community_model()

Set parameter Username
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "leghorn"


Output()

Output()

Read LP format model from file C:\Users\pino216\AppData\Local\Temp\tmpxuplwx7z.lp
Reading time = 0.07 seconds
: 5454 rows, 12699 columns, 49791 nonzeros


## Optimize Model

In [ ]:
result = community.optimize(fluxes=True, pfba=True)
result

### Update Medium

In [ ]:
# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_sucr_m"] = 5
medium_to_change["EX_glc__D_m"] = 5
medium_to_change["EX_leu__L_m"] = 0
#medium_to_change["EX_glyc__R_m"] = 0
medium_to_change["EX_nh4_m"] = 0
medium_to_change["EX_no3_m"] = 0
medium_to_change["EX_n2_m"] = 5


# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

In [ ]:
result_altered_medium = community.optimize(fluxes=True, pfba=True)

In [ ]:
result_altered_medium